In [30]:
#we need to use curl command to download the dataset from the roboflow and also specifying the unique id
!curl -L "your key" > roboflow.zip; unzip roboflow.zip; rm roboflow.zip

curl: (3) Host name 'your key' contains bad letter
Archive:  roboflow.zip
  End-of-central-directory signature not found.  Either this file is not
  a zipfile, or it constitutes one disk of a multi-part archive.  In the
  latter case the central directory and zipfile comment will be found on
  the last disk(s) of this archive.
unzip:  cannot find zipfile directory in one of roboflow.zip or
        roboflow.zip.zip, and cannot find roboflow.zip.ZIP, period.


In [2]:
#we will use a pretrained MobileNetV2 classifier for custom requirement
#The pretrained models are available in the library torchvision.models

from torchvision import models
from torchsummary import summary
import torch
from torch import nn
from torch.optim import Adam

In [3]:
custom_model = models.mobilenet_v2(pretrained=True)

Downloading: "https://download.pytorch.org/models/mobilenet_v2-b0353104.pth" to /root/.cache/torch/hub/checkpoints/mobilenet_v2-b0353104.pth


  0%|          | 0.00/13.6M [00:00<?, ?B/s]

In [7]:
#The next step is to findout the name of the last layer where we can replace the parameters according to our requirement
#for that we can use named_children() function which displays the name of the layers

for name, model in custom_model.named_children():
  print(name)

features
classifier


In [19]:
#The main blocks are features and classifiers
#Let's display the individual components

custom_model.classifier

Sequential(
  (0): Dropout(p=0.2, inplace=False)
  (1): Linear(in_features=1280, out_features=1000, bias=True)
)

In [20]:
#similar way we can check the features as well
#features comprise of the convolutional, maxpooling blocks

In [21]:
#since the above corrsponds to imagenet setup, we can see the last layer has 1000 classes
#And also we can either freeze the already trained model layers or unfreeze them
#This can be done by giving the command requires_grad = False, means layers are frozen
#for the above there is no training happens

for param in custom_model.parameters():
  param.requires_grad = False

#since it is a binary classification, the output will have two classes 

#if we have a GPU available, then we can use cuda
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

device

device(type='cuda')

In [22]:
num_ftrs = custom_model.classifier[1].in_features
print('number of input features to the last linear layer:', num_ftrs)

#now we need to update the last layer to output only two classes
custom_model.classifier[1] = nn.Linear(num_ftrs, 2)

number of input features to the last linear layer: 1280


In [24]:
#lets print the last layer and see whether the update has been reflected
custom_model.classifier

Sequential(
  (0): Dropout(p=0.2, inplace=False)
  (1): Linear(in_features=1280, out_features=2, bias=True)
)

From the above definition we can see the output size is 2 corresponding to two categories.

In [25]:
#converting to cuda format
custom_model =  custom_model.to(device)

In [26]:
#setting up dataloader(multiple smaller batches). Since we do not process all the training
#samples at one go, we use dataloaders for processing multiple data chunks
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torchvision.transforms import ToTensor, Resize

In [27]:
#define the path details from where the images should be retried
train_dataset = datasets.ImageFolder(root = '/content/train', transform = transforms.Compose([Resize((224,224)),ToTensor()]))
val_dataset = datasets.ImageFolder(root='/content/valid', transform=transforms.Compose([Resize((224,224)),ToTensor()]))

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size = 32, shuffle=True)

In [28]:
#for loss function we can use the cross entropy function
loss_func = nn.CrossEntropyLoss()
opt = Adam(custom_model.parameters(), lr = 0.001)

In [29]:
epochs = 10

for e in range(0, epochs):
  custom_model.train()

  totalTrainLoss = 0
  totalValLoss = 0

  trainCorrect = 0
  valCorrect = 0

  for (x, y) in train_loader:
    (x,y) = (x.to(device), y.to(device))

    pred = custom_model(x)
    loss = loss_func(pred, y)

    opt.zero_grad()
    loss.backward()
    opt.step()

    totalTrainLoss += loss
    trainCorrect += (pred.argmax(1) == y).type(torch.float).sum().item()

  print('Train loss for the epoch'+ str(e)+'is:',totalTrainLoss)
  print('Number of correctly predicted outputs for the epoch' + str(e)+ 'is:', trainCorrect)

Train loss for the epoch0is: tensor(15.8482, device='cuda:0', grad_fn=<AddBackward0>)
Number of correctly predicted outputs for the epoch0is: 1062.0
Train loss for the epoch1is: tensor(8.6196, device='cuda:0', grad_fn=<AddBackward0>)
Number of correctly predicted outputs for the epoch1is: 1175.0
Train loss for the epoch2is: tensor(7.4713, device='cuda:0', grad_fn=<AddBackward0>)
Number of correctly predicted outputs for the epoch2is: 1177.0
Train loss for the epoch3is: tensor(6.5840, device='cuda:0', grad_fn=<AddBackward0>)
Number of correctly predicted outputs for the epoch3is: 1198.0
Train loss for the epoch4is: tensor(6.2062, device='cuda:0', grad_fn=<AddBackward0>)
Number of correctly predicted outputs for the epoch4is: 1193.0
Train loss for the epoch5is: tensor(5.6925, device='cuda:0', grad_fn=<AddBackward0>)
Number of correctly predicted outputs for the epoch5is: 1201.0
Train loss for the epoch6is: tensor(5.0672, device='cuda:0', grad_fn=<AddBackward0>)
Number of correctly predic